# Lab. 군집화

## Clustering
+ k-means Clustering
+ Hierarchical Clustering

## Evaluation
+ Silhouette

In [ ]:
import os
from os.path import join
import copy
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sklearn

import matplotlib.pyplot as plt

- 이번 군집화 실습을 위해 sklearn 내장 데이터인 와인 Dataset을 불러온다.
- 와인 Dataset은 알콜, 말산, 페놀 등 13개의 변수를 가지고 있으며, 1,2,3 와인 등급을 label data로 가지고 있다.

In [ ]:
from sklearn.datasets import load_wine
wine = load_wine()

In [ ]:
print(wine.DESCR)

In [ ]:
data = wine.data
label = wine.target
columns = wine.feature_names

In [ ]:
data = pd.DataFrame(data, columns = columns)
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

## Clustering
- 주어진 데이터들의 특성을 고려해 Data Cluster를 정의하고, Cluster를 대표할 수 있는 대표점을 찾는 비지도 학습의 대표적인 Algorithm.
- 간단히 말해서, 비슷한 특성을 가진 데이터끼리 묶는다고 말할 수 있다. 

### 1. k-Means Clustering
- k-means Clustering은 대표적인 Clustering Algorithm 중 하나로, 각 Cluster에 할당된 Data Point들의 평균 좌표를 이용해 군집중심점(centroid)을 반복적으로 update하며 Cluster를 형성하는 Algorithm이다.
- 군집 중심점은 선택된 point의 평균 지점으로 이동하고 이동된 중심점에서 다시 가까운 point를 선택, 다시 중심점을 평균 지점으로 이동하는 process를 반복적으로 수행한다.
- 모든 data point에서 더 이상 중심점의 이동이 없을 경우에 반복을 멈추고 해당 중심점에 속하는 data point들을 군집화하는 기법이다.

 ![./Images/k-means-clustering.png](./img/k-means-clustering.png)
- k-means Clustering Algorithm은 3가지 단계로 이루어진다.<br />
Step 1. 각 Data Point i 에 대해 가장 가까운 중심점을 찾고, 그 중심점에 해당하는 Cluster를 할당.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;가까운 중심점을 찾을 때는, **유클리드 거리** 를 사용.<br>
Step 2. 할당된 Cluster를 기반으로 새로운 중심점을 계산.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;중심점은 Cluster 내부 점들 좌표의 **산술 평균(mean)** 으로 한다.<br>
Step 3. 각 Cluster의 할당이 바뀌지 않을 때까지 반복한다.

 ### 점과 점사이의 거리를 어떻게 측정할 수 있을까? 

 k-means Clustering은 거리 기반 Algorithm이므로 여러가지 방법으로 거리를 측정할 수 있다.<br>
 #### 1. Manhattan Distance - 각 축에 대해 수직으로만 이동하여 계산하는 거리 측정방식
 $$D(x,y) = {{\sum_{i=1}^{d}  |x_i - y_i|} } $$
 ![./Images/Manhattan.png](./img/Manhattan.png)
 
 #### 2. Euclidean Distance - 점과 점사이의 가장 짧은 거리를 계산하는 거리 측정방식
 $$D(x,y) = {\sqrt{\sum_{i=1}^{d}  (x_i - y_i)^2} } $$
 ![./Images/Euclidean.png](./img/Euclidean.png)
 

## 준비
- wine Data는 13개의 column을 가지고 있고, 하나의 데이터(행)는 13개의 차원으로 이루어진 vector라고 볼 수 있다. 
- 13차원은 우리 눈으로 볼 수 있도록 표현하기 어려우므로 앞에서 배운 pca를 통해 2차원으로 만들어 시각화할 수 있도록 변환하자.
- 그 전에 각 변수들의 값의 범위가 서로 다르므로 min-max 정규화를 통해 조정해 준다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
data = pca.fit_transform(data)

In [ ]:
data.shape

#### 1) Model 불러오기 및 정의하기
- Clustering은 비지도학습이므로 Cluster의 수는 label의 수와 관계 없지만, 3개의 군집을 형성하도록 한다.
- k-means Clustering은 sklearn의 cluster package에 있다. 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)

#### 2) Model 학습하기 (Clustering을 통한 중심점 찾기)

In [ ]:
kmeans.fit(data)

#### 3) Cluster 할당

In [ ]:
cluster = kmeans.predict(data)

#### 4) 결과 살펴보기

In [ ]:
plt.scatter(data[:, 0], data[:, 1], c=cluster, linewidth=1, edgecolor='black')
plt.show()

## K-Means를 이용한 붓꽃(Iris) 데이터 셋 Clustering

In [ ]:
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

iris = load_iris()
# 보다 편리한 데이터 Handling을 위해 DataFrame으로 변환
irisDF = pd.DataFrame(data=iris.data, columns=['sepal_length','sepal_width','petal_length','petal_width'])
irisDF.head(3)

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300,random_state=0).fit(irisDF)
# init : 초기에 군집 중심점의 좌표를 설정할 방식을 말하며, 보통은 임의로 중심을 설정하지 않고 일반적으로 k-means++ 방식으로 최초 설정한다.

In [ ]:
print(kmeans.labels_)

In [ ]:
irisDF['cluster'] = kmeans.labels_

In [ ]:
irisDF.head()

In [ ]:
irisDF['target'] = iris.target
iris_result = irisDF.groupby(['target','cluster'])['sepal_length'].count()
print(iris_result)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(iris.data)

irisDF['pca_x'] = pca_transformed[:,0]
irisDF['pca_y'] = pca_transformed[:,1]
irisDF.head(3)

In [ ]:
# cluster 값이 0, 1, 2 인 경우마다 별도의 Index로 추출
marker0_ind = irisDF[irisDF['cluster']==0].index
marker1_ind = irisDF[irisDF['cluster']==1].index
marker2_ind = irisDF[irisDF['cluster']==2].index

# cluster값 0, 1, 2에 해당하는 Index로 각 cluster 레벨의 pca_x, pca_y 값 추출. o, s, ^ 로 marker 표시
plt.scatter(x=irisDF.loc[marker0_ind,'pca_x'], y=irisDF.loc[marker0_ind,'pca_y'], marker='o') 
plt.scatter(x=irisDF.loc[marker1_ind,'pca_x'], y=irisDF.loc[marker1_ind,'pca_y'], marker='s')
plt.scatter(x=irisDF.loc[marker2_ind,'pca_x'], y=irisDF.loc[marker2_ind,'pca_y'], marker='^')

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('3 Clusters Visualization by 2 PCA Components')
plt.show()

### K-Means의 장점
- 일반적인 군집화에서 가장 많이 활용되는 Algorithm이다.
- Algorithm이 쉽고 간결하다.

### K-Means의 단점
- 거리 기반 Algorithm으로 속성의 갯수가 매우 많을 경우 군집화 정확도가 떨어진다. 그래서 PCA로 차원 감소를 적용해야 할 수 있다.
- 반복을 수행하는데, 반복 횟수가 많을 경우 수행 시간이 매우 느려진다.
- 몇 개의 군집(Cluster)을 선택해야 할지 가이드하기 어렵다.

## 2. Hierarchical Clustering
- 거리(Distance) 또는 유사도(Similarity)를 기반으로 Cluster를 형성하는 Algorithm 이다. 
- k-means Clustering과 다르게 Cluster의 수를 설정해 줄 필요가 없으며, Cluster 형태를 시각적으로 표현해주는 Dendrogram을 통해 적절한 Cluster의 수를 선택할 수 있다.
- Dendrogram : 각 단계에서 관측치의 군집화를 통해 형성된 group과 이들의 유사성 수준을 표시하는 Tree Diagram.
- Hierachichal Clustering에는 Bottom-Up 방식의 Agglomerative Method와 Top-Down 방식의 Divisive Method로 나뉜다.<br /><br />
여기에서는 Agglomerative Method를 사용해 실습을 진행한다.
<br><br>Agglomerative Method를 사용한 Hierarchical Clustering Algorithm은 3가지 단계로 이루어진다.<br>
- Step 1. 각 Data Point를 Cluster로 할당한다. (n개의 Cluster)
- Step 2. 가까운 Cluster끼리 병합한다.
- Step 3. 1개의 Cluster가 될 때까지 반복한다.

### 어떻게 가장 가까운 클러스터를 찾을 수 있을까?
- 방금전 거리 측정 방법으로 맨하탄 거리, 유클리디언 거리에 대해 배웠었다.
- k-means에서는 각 Cluster의 중심점 간의 거리로 Cluster간 거리를 계산했었다.

<br />이번 수업에서는 새로운 Cluster간 거리를 계산하는 방법에 대해 알아보도록 한다.<br>
#### 1. Single Linkage - 두 Cluster 내의 가장 가까운 점 사이의 거리 
![Single Linkage](./img/Single.png)
#### 2. Complete Linkage - 두 Cluster 내의 가장 먼 점 사이의 거리
![Complete Linkage](./img/Complete.png)
#### 3. Average Linkage - 두 Cluster 내의 모든 점 사이의 평균 거리
![Average Linkage](./img/Average.png)

#### 3개 거리 측정 방식의 결과와 차이점을 살펴보자.

### Single Linkage

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.cluster import AgglomerativeClustering
single_clustering = AgglomerativeClustering(n_clusters=3, linkage='single')

#### 2) 모델 학습하기 (클러스터링을 통한 중심점 찾기)

In [ ]:
single_clustering.fit(data)

#### 3) 클러스터 할당

In [ ]:
single_cluster = single_clustering.labels_

#### 4) 결과 살펴보기
* 산점도

In [ ]:
plt.scatter(data[:,0], data[:,1], c=single_cluster)
plt.title('Sklearn Single Linkage Hierarchical Clustering')
plt.show()

- dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram
plt.figure(figsize=(10,10))

# Hierarchical Clustering의 자식 노드
children = single_clustering.children_

# 각 자식 노드간의 거리 정보를 가지고 있지 않기 때문에, 균일하게 그리도록 한다.
distance = np.arange(children.shape[0])

# 각 Cluster 단계를 포함한 노드의 수 계산
no_of_observations = np.arange(2, children.shape[0]+2)

# Dendrogram을 그리기위한 연결 Metrix 생성.
linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

# Dendrogram을 그린다.
dendrogram(linkage_matrix, p = len(data), labels = single_cluster, 
           show_contracted=True, no_labels = True, )
plt.show()

### Complete Linkage

#### 1) Model 불러오기 및 정의하기

In [ ]:
complete_clustering = AgglomerativeClustering(n_clusters=3, linkage='complete')

#### 2) Model 학습하기 (Clustering을 통한 중심점 찾기)

In [ ]:
complete_clustering.fit(data)

#### 3) 클러스터 할당

In [ ]:
complete_cluster = complete_clustering.labels_

#### 4) 결과 살펴보기
* 산점도

In [ ]:
plt.scatter(data[:,0], data[:,1], c=complete_cluster)
plt.title('Sklearn Complete Linkage Hierarchical Clustering')
plt.show()

- Dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram
plt.figure(figsize=(10,10))

# Hierarchical Clustering의 자식 노드
children = complete_clustering.children_

# 각 자식 노드간의 거리 정보를 가지고 있지 않기 때문에, 균일하게 그리도록 한다.
distance = np.arange(children.shape[0])

# 각 Cluster 단계를 포함한 노드의 수 계산
no_of_observations = np.arange(2, children.shape[0]+2)

# Dendrogram을 그리기위한 연결 Matrix를 생성합니다.
linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

# Dendrogram을 그립니다.
dendrogram(linkage_matrix, p = len(data), labels = complete_cluster, 
           show_contracted=True, no_labels = True, )
plt.show()

### Average Linkage

#### 1) Model 불러오기 및 정의하기

In [ ]:
average_clustering = AgglomerativeClustering(n_clusters=3, linkage='average')

#### 2) Model 학습하기 (Clustering을 통한 중심점 찾기)

In [ ]:
average_clustering.fit(data)

#### 3) Clustering 할당

In [ ]:
average_cluster = average_clustering.labels_

#### 4) 결과 살펴보기
* 산점도

In [ ]:
plt.scatter(data[:,0], data[:,1], c=average_cluster)
plt.title('Sklearn Average Linkage Hierarchical Clustering')
plt.show()

- Dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram
plt.figure(figsize=(10,10))

# Hierarchical Clustering의 자식 노드
children = average_clustering.children_

# 각 자식 노드간의 거리 정보를 가지고 있지 않기 때문에, 균일하게 그리도록 한다.
distance = np.arange(children.shape[0])

# 각 Cluster 단계를 포함한 노드의 수 계산
no_of_observations = np.arange(2, children.shape[0]+2)

# Dendrogram을 그리기위한 연결 Matrix 생성.
linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

# Dendrogram을 그린다.
dendrogram(linkage_matrix, p = len(data), labels = average_cluster, 
           show_contracted=True, no_labels = True, )
plt.show()

### Clustering 결과 비교하기
1. Single Linkage
    + 두 Cluster 내의 가장 가까운 점을 기준으로 Cluster를 합치기 때문에 Cluster 사이의 Noise에 매우 민감한 특성과 구 형태가 아닌 data에 대해 Cluster를 잘 형성한다는 특성이 있다.
    + wine data는 모든 data가 연결되어 있는 듯한 분포를 가지고 있기 때문에, 각 Cluster의 경계가 모호한 Noise가 많은 형태를 띠고 있다. <br>Single Linkage가 구 형태가 아닌 data에 대해 Cluster를 잘 형성한다는 특성이 있지만, 이러한 data의 경우 Single Linkage 방법을 사용하면 좋은 Cluster를 생성하기 어렵다.

In [ ]:
plt.figure(figsize=(5,4))

plt.scatter(data[:,0], data[:,1], c=single_cluster)
plt.title('Sklearn Single Linkage Hierarchical Clustering')
plt.show()

2. Complete Linkage
    + 두 Cluster 내에 가장 먼 점을 기준으로 Cluster를 합치기 때문에 Cluster 사이의 Noise와 이상치에 민감하지 않은 특성이 있다.
    + Noise에 민감하지 않다는 특성을 가진 Complete Linkage가 좋은 성능을 보여주었다.

3. Average Linkage
    + Single Linkage와 Complete Linkage의 중간쯤에 위치한 Average Linkage가 가장 정답에 가까운 Cluster를 형성한 것을 확인할 수 있다. 

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1, 2, 1)
plt.scatter(data[:,0], data[:,1], c=complete_cluster)
plt.title('Sklearn Complete Linkage Hierarchical Clustering')

plt.subplot(1, 2, 2)
plt.scatter(data[:,0], data[:,1], c=average_cluster)
plt.title('Sklearn Average Linkage Hierarchical Clustering')
plt.show()

## Evaluation
### Silhouette
+ Silhouette 값은 한 cluster 안의 데이터들이 다른 cluster와 비교해서 얼마나 비슷한가를 나타냅니다.
+ 다시 말해서 각 군집 간의 거리가 얼마나 효율적으로 분리돼 있는지를 나타낸다.
+ 같은 cluster 내의 점들간 거리는 가깝고(cohesion) 서로 다른 cluster 간의 거리는 멀수록(separation) 높은 값을 얻을 수 있다.
+ Silhouette 값이 1에 근접한다는 것은 같은 cluster 내의 평균거리가 다른 cluster와의 평균거리보다 가깝다는 것을 의미한다.
+ 일반적으로 Silhouette 값이 0.5보다 크다면 데이터가 잘 clustering 되었다는 것을 나타낸다.

Silhouette 공식은 다음과 같다.
$$ S_i = { {(b_i - a_i)} \over max(a_i, b_i) }$$

$$ a_i\ :\ 같은\ 클러스터\ 내의\ 모든\ 점들\ 간\ 평균\ 거리 $$
$$ b_i\ :\ \bar d\ =\ (i,c)의\ 최솟값 $$
$$ \bar d\ =\ (i,c)\ :\ 다른\ 클러스터\ c와\ i번째 데이터 와의\ 평균\ 거리$$
<br>

- 직관적으로 수식을 이해해보자.
- a<sub>i</sub>는 같은 Cluster 내의 데이터 들이 잘 모여있다면 적은 값을 나타내고, b<sub>i</sub>는 각 Cluster들이 멀리 떨어져있다면 큰 값을 나타내게 된다.
- 따라서 수식 S<sub>i</sub>에 따르면, 아주 잘 형성된(같은 Cluster는 가깝고 다른 Cluster끼리는 먼) Cluster 형태일 때 분모는 b<sub>i</sub>이 되고,<br> 분자는 b<sub>i</sub>에서 아주 작은 값인 a<sub>i</sub>가 빠져 1에 가까운 silhouette 값을 얻을 수 있다.

#### 가장 좋은 Cluster를 형성하는 Cluster의 수를 찾아보자.
- k-means Clustering과 Average Linkage를 사용한 Hierarchical Clustering에서 가장 높은 점수의 Cluster 수는 무엇인지 알아본다.

- Silhouette Scoring은 Sklearn의 metrics 패키지에 있다.

#### 1) k-means

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
best_n = 1
best_score = -1

for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster)
    kmeans.fit(data)
    cluster = kmeans.predict(data)
    score = silhouette_score(data, cluster)
    
    print('클러스터의 수 : {}, 실루엣 점수 : {:.2f}'.format(n_cluster, score))
    if score > best_score :
        best_n = n_cluster
        best_score = score
        
print('가장 높은 실루엣 점수를 가진 클러스터 수 : {}, 실루엣 점수 : {:.2f}'.format(best_n, best_score))

#### 2) Average Linkage Hierarchical Clustering

In [ ]:
from sklearn.metrics import silhouette_score

best_n = 1
best_score = -1

for n_cluster in range(2, 11):
    average_clustering = AgglomerativeClustering(n_clusters= n_cluster, linkage='average')
    average_clustering.fit(data)
    cluster = average_clustering.labels_
    score = silhouette_score(data, cluster)
    
    print('클러스터의 수 : {}, 실루엣 점수 : {:.2f}'.format(n_cluster, score))
    if score > best_score :
        best_n = n_cluster
        best_score = score
        
print('가장 높은 실루엣 점수를 가진 클러스터 수 : {}, 실루엣 점수 : {:.2f}'.format(best_n, best_score))



### Reference
- Wikipedia, Clustering : https://ko.wikipedia.org/wiki/클러스터_분석
- Wikipedia, Manhattan distance : https://ko.wikipedia.org/wiki/맨해튼_거리
- Wikipedia, Euclidean distance : https://ko.wikipedia.org/wiki/유클리드_거리
- Sklearn, Wine dataset : https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html
- Sklearn, k-Means Clustering : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
- Sklearn, Hierarchical Clustering : https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html&sa=U&ved=0ahUKEwj_2aiGvt7hAhXLi7wKHei8CNsQFggEMAA&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw0zVZAVTxgORo-7LbgrNv_o
- Sklearn, Silhouette : https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html&sa=U&ved=0ahUKEwi5lrTZwd7hAhUqCqYKHWCZCTEQFggEMAA&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw0-ZT8AJZRmR-qNpN-62Ei-
- Sklearn, Silhouette Example : https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
- Scipy, Dendrogram : https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.dendrogram.html